In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Берем выборку

In [12]:
# dataset = load_breast_cancer()
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']
# df = (df - df.mean())/(df.max() - df.min())
# df0 = df.copy()
# print(df.shape)
# print(target)
df.head()

# train/test split
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.33, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, X_val.shape)

(1005, 64) (360, 64) (432, 64)


In [16]:
dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)

Training dich classifiers:  70%|███████   | 21/30 [00:00<00:00, 45.91it/s]


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

# N случайных дихотомий (без отбора)

In [114]:
attempts_data = []
N_attempts = 50
l = np.unique(target).size
N = 100 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
wtypes = ['confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        weight_type = None
        preds = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))









  0%|          | 0/50 [00:00<?, ?it/s]








Adding dich:   0%|          | 0/100 [00:00<?, ?it/s]








Adding dich: 100%|██████████| 100/100 [00:00<00:00, 35359.16it/s]








Training dich classifiers:   0%|          | 0/100 [00:00<?, ?it/s]








Training dich classifiers:   4%|▍         | 4/100 [00:00<00:02, 36.38it/s]

Code Matrix shape == (10,100)











Training dich classifiers:   8%|▊         | 8/100 [00:00<00:02, 36.77it/s]








Training dich classifiers:  13%|█▎        | 13/100 [00:00<00:02, 39.14it/s]








Training dich classifiers:  18%|█▊        | 18/100 [00:00<00:01, 41.41it/s]








Training dich classifiers:  24%|██▍       | 24/100 [00:00<00:01, 43.54it/s]








Training dich classifiers:  29%|██▉       | 29/100 [00:00<00:01, 44.38it/s]








Training dich classifiers:  35%|███▌      | 35/100 [00:00<00:01, 45.94it/s]








Training dich classifiers:  40%|████      | 40/100 [00:00<00:01, 46.29it/s]








Training dich classifiers:  45%|████▌     | 45/100 [00:00<00:01, 46.40it/s]








Training dich classifiers:  50%|█████     | 50/100 [00:01<00:01, 45.83it/s]








Training dich classifiers:  55%|█████▌    | 55/100 [00:01<00:00, 45.41it/s]








Training dich classifiers:  60%|██████    | 60/100 [00:01<00:00, 45.73it/s]








Training dich classifiers:  65%|██████▌   | 65/100 [00:01<00:00, 46.

Code Matrix shape == (10,100)











Training dich classifiers:  10%|█         | 10/100 [00:00<00:02, 43.87it/s]








Training dich classifiers:  16%|█▌        | 16/100 [00:00<00:01, 45.69it/s]








Training dich classifiers:  21%|██        | 21/100 [00:00<00:01, 46.02it/s]








Training dich classifiers:  26%|██▌       | 26/100 [00:00<00:01, 46.11it/s]








Training dich classifiers:  31%|███       | 31/100 [00:00<00:01, 46.40it/s]








Training dich classifiers:  36%|███▌      | 36/100 [00:00<00:01, 45.98it/s]








Training dich classifiers:  41%|████      | 41/100 [00:00<00:01, 46.24it/s]








Training dich classifiers:  46%|████▌     | 46/100 [00:00<00:01, 46.10it/s]








Training dich classifiers:  51%|█████     | 51/100 [00:01<00:01, 45.94it/s]








Training dich classifiers:  57%|█████▋    | 57/100 [00:01<00:00, 46.52it/s]








Training dich classifiers:  62%|██████▏   | 62/100 [00:01<00:00, 46.46it/s]








Training dich classifiers:  67%|██████▋   | 67/100 [00:01<00:00, 46

...










Training dich classifiers:  10%|█         | 10/100 [00:00<00:01, 45.35it/s]








Training dich classifiers:  15%|█▌        | 15/100 [00:00<00:01, 45.73it/s]








Training dich classifiers:  20%|██        | 20/100 [00:00<00:01, 46.12it/s]








Training dich classifiers:  25%|██▌       | 25/100 [00:00<00:01, 45.38it/s]








Training dich classifiers:  30%|███       | 30/100 [00:00<00:01, 46.04it/s]








Training dich classifiers:  35%|███▌      | 35/100 [00:00<00:01, 45.59it/s]








Training dich classifiers:  40%|████      | 40/100 [00:00<00:01, 45.99it/s]








Training dich classifiers:  45%|████▌     | 45/100 [00:00<00:01, 45.48it/s]








Training dich classifiers:  50%|█████     | 50/100 [00:01<00:01, 45.47it/s]








Training dich classifiers:  55%|█████▌    | 55/100 [00:01<00:00, 45.38it/s]








Training dich classifiers:  60%|██████    | 60/100 [00:01<00:00, 45.10it/s]








Training dich classifiers:  65%|██████▌   | 65/100 [00:01<00:00, 45

Code Matrix shape == (10,100)











Training dich classifiers:   9%|▉         | 9/100 [00:00<00:02, 40.76it/s]








Training dich classifiers:  14%|█▍        | 14/100 [00:00<00:02, 41.96it/s]








Training dich classifiers:  19%|█▉        | 19/100 [00:00<00:01, 42.95it/s]








Training dich classifiers:  24%|██▍       | 24/100 [00:00<00:01, 43.55it/s]








Training dich classifiers:  29%|██▉       | 29/100 [00:00<00:01, 43.85it/s]








Training dich classifiers:  34%|███▍      | 34/100 [00:00<00:01, 44.11it/s]








Training dich classifiers:  39%|███▉      | 39/100 [00:00<00:01, 44.74it/s]








Training dich classifiers:  44%|████▍     | 44/100 [00:00<00:01, 44.45it/s]








Training dich classifiers:  49%|████▉     | 49/100 [00:01<00:01, 44.18it/s]








Training dich classifiers:  54%|█████▍    | 54/100 [00:01<00:01, 44.03it/s]








Training dich classifiers:  59%|█████▉    | 59/100 [00:01<00:00, 44.28it/s]








Training dich classifiers:  64%|██████▍   | 64/100 [00:01<00:00, 44.

Code Matrix shape == (10,100)











Training dich classifiers:  11%|█         | 11/100 [00:00<00:01, 46.43it/s]








Training dich classifiers:  16%|█▌        | 16/100 [00:00<00:01, 45.86it/s]








Training dich classifiers:  21%|██        | 21/100 [00:00<00:01, 45.96it/s]








Training dich classifiers:  26%|██▌       | 26/100 [00:00<00:01, 46.25it/s]








Training dich classifiers:  31%|███       | 31/100 [00:00<00:01, 45.22it/s]








Training dich classifiers:  36%|███▌      | 36/100 [00:00<00:01, 45.30it/s]








Training dich classifiers:  41%|████      | 41/100 [00:00<00:01, 45.35it/s]








Training dich classifiers:  46%|████▌     | 46/100 [00:01<00:01, 45.50it/s]








Training dich classifiers:  51%|█████     | 51/100 [00:01<00:01, 45.13it/s]








Training dich classifiers:  56%|█████▌    | 56/100 [00:01<00:00, 45.24it/s]








Training dich classifiers:  61%|██████    | 61/100 [00:01<00:00, 45.18it/s]








Training dich classifiers:  66%|██████▌   | 66/100 [00:01<00:00, 45

In [7]:
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}_w{}'.format(i1, i2) for i1 in wtypes for i2 in wtypes]
df_attempts

,sNone_wNone,sNone_waccuracy,sNone_wf1,sNone_wconfusion_list,saccuracy_wNone,saccuracy_waccuracy,saccuracy_wf1,saccuracy_wconfusion_list,sf1_wNone,sf1_waccuracy,sf1_wf1,sf1_wconfusion_list,sconfusion_list_wNone,sconfusion_list_waccuracy,sconfusion_list_wf1,sconfusion_list_wconfusion_list
0,0.875000,0.872685,0.868056,0.865741,0.886574,0.886574,0.884259,0.886574,0.884259,0.884259,0.884259,0.884259,0.870370,0.851852,0.851852,0.851852
1,0.893519,0.891204,0.893519,0.886574,0.886574,0.886574,0.886574,0.886574,0.888889,0.888889,0.888889,0.888889,0.909722,0.824074,0.824074,0.824074
2,0.893519,0.888889,0.888889,0.891204,0.900463,0.900463,0.900463,0.900463,0.902778,0.902778,0.902778,0.902778,0.905093,0.796296,0.796296,0.796296
3,0.895833,0.898148,0.893519,0.886574,0.891204,0.891204,0.891204,0.891204,0.895833,0.895833,0.895833,0.895833,0.900463,0.833333,0.833333,0.833333
4,0.886574,0.893519,0.895833,0.888889,0.898148,0.898148,0.898148,0.898148,0.900463,0.900463,0.900463,0.900463,0.900463,0.831019,0.831019,0.831019
5,0.909722,0.912037,0.909722,0.912037,0.914352,0.914352,0.914352,0.914352,0.914352,0.914352,0.914352,0.914352,0.912037,0.881944,0.881944,0.881944
6,0.891204,0.891204,0.888889,0.891204,0.895833,0.895833,0.895833,0.895833,0.895833,0.895833,0.895833,0.895833,0.902778,0.842593,0.842593,0.842593
7,0.902778,0.900463,0.895833,0.902778,0.907407,0.907407,0.907407,0.907407,0.907407,0.907407,0.907407,0.907407,0.898148,0.861111,0.861111,0.861111
8,0.902778,0.893519,0.898148,0.900463,0.902778,0.902778,0.902778,0.902778,0.900463,0.900463,0.900463,0.900463,0.907407,0.814815,0.814815,0.814815
9,0.895833,0.898148,0.898148,0.898148,0.905093,0.905093,0.905093,0.905093,0.902778,0.902778,0.902778,0.902778,0.905093,0.861111,0.861111,0.861111


In [ ]:
df_attempts.describe()

In [ ]:
df_attempts

In [ ]:
x = np.array([1,1,1])
np.random.choice(np.flatnonzero(x == x.max()))

# Локальный метод оптимизации

In [ ]:
def score_function(cur_dich, X_train, y_train, X_test, y_test):
    # cluster
    target = np.array([cur_dich[i] for i in y_train])
    return -functions.cluster_score(X_train, target, score_type='trace_w')

In [ ]:
def score_function(cur_dich, X_train, y_train, X_test, y_test):
    # accuracy
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [5]:
def score_function(cur_dich, X_train, y_train, X_test, y_test):
    # f1
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = f1_score(y_true, y_pred)
    return accuracy

In [17]:
cur_dich = code_matrix.T[1].copy()
cur_score = score_function(cur_dich, X_train, y_train, X_test, y_test)
next_score = cur_score
cur_score

1.0

In [14]:
while True:
    next_dich = cur_score.copy()
    next_scores = np.zeros(len(cur_dich)) - 1
    for i in range(len(cur_dich)):
        next_dich = cur_dich.copy()
        next_dich[i] = 1 - next_dich[i]
        if not functions.does_dich_exist(next_dich, code_matrix): #дихотомия нормальная
            next_scores[i] = score_function(next_dich, X_train, y_train, X_test, y_test)
    next_scores = np.array(next_scores)
    next_score = next_scores.max()
    if next_score <= cur_score: #идем только на повышение, но можно скор сделать поменьше
        break
    cur_score = next_score
    best_index = np.random.choice(np.flatnonzero(next_scores == next_score)) # it is random of the best
    print(next_score, best_index)
    print(cur_dich)
    cur_dich[best_index] = 1 - cur_dich[best_index]

0.9714285714285714 9
[0 0 0 0 0 0 1 0 0 1]
0.9753086419753086 4
[0 0 0 0 0 0 1 0 0 0]
1.0 6
[0 0 0 0 1 0 1 0 0 0]


In [15]:
cur_dich

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])


# Конструируем модельную задачу

In [ ]:
np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, num_clusters-1, num_clusters):
    for j in np.linspace(0, num_clusters-1, num_clusters):
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
X = np.vstack(X)
scatter(X[:,0], X[:,1], )

In [ ]:
# объединяем классы
classes_in_cluster = 5

order = np.array(range(num_clusters**2))
np.random.shuffle(order)
splits = np.split(order, 5)
for i, split in enumerate(splits):
    for item in split:
        y[item*cluster_objects:(item+1)*cluster_objects] = i

In [ ]:
print(y)
plot(y)

In [ ]:
figsize(8, 8)
scatter(X[:,0], X[:,1], c=y, )

In [ ]:
# модельная задача на 12 классов
colors = []
cs = []

# np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, 4-1, 4):
    for j in np.linspace(0, 5-1, 5):
        c = np.random.randint(0, 12)
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
        colors += [c]*cluster_objects
        cs.append(c)
X = np.vstack(X)
print(cs)
scatter(X[:,0], X[:,1], c=colors)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_similarity_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_mldata

# Load a multi-label dataset
yeast = fetch_mldata('yeast')
X = yeast['data']
Y = yeast['target'].transpose().toarray()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2,
                                                    random_state=0)

In [ ]:
yeast['target']

In [ ]:
df = pd.read_csv('../yeast.data.txt', sep=';', header=None)
X = df.values[:,1:-1]
y = pd.factorize(df[9])[0]
y

In [ ]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)

In [ ]:
scatter(tsne_results[:,0], tsne_results[:,1], c=y)
